In [28]:
import warnings
warnings.filterwarnings("ignore")
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere, get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data
from dipy.reconst import sfm
from dipy.reconst.csdeconv import auto_response_ssst
from dipy.tracking import utils
from numpy.linalg import inv
%load_ext line_profiler
%load_ext memory_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [2]:
hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')
label_fname = get_fnames('stanford_labels')

data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
labels = load_nifti_data(label_fname)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

In [3]:
white_matter = (labels == 1) | (labels == 2)

In [4]:
response, ratio = auto_response_ssst(gtab, data, fa_thr=0.2)

In [23]:
sphere = get_sphere()
sfmodel = sfm.SparseFascicleModel(gtab, sphere=sphere, l1_ratio=0.5, alpha=0.001, response=response[0])

In [29]:
n_procs_list = [1, 2, 4, 8]

def profile_fit(sfmodel, data, white_matter, n_procs):
    print(f"CPUs: {n_procs}")
    %memit sfmodel.fit(data, mask=white_matter, num_processes=n_procs)
    return

def loop_runs(sfmodel, data, white_matter, n_procs_list):
    for n_procs in n_procs_list:
        %time profile_fit(sfmodel, data, white_matter, n_procs)
    return

loop_runs(sfmodel, data, white_matter, n_procs_list)

CPUs: 1
peak memory: 1932.80 MiB, increment: 1805.14 MiB
CPU times: user 1h 19min 48s, sys: 15min 29s, total: 1h 35min 18s
Wall time: 18min 5s
CPUs: 2
peak memory: 2223.09 MiB, increment: 2031.77 MiB
CPU times: user 1h 19min 50s, sys: 13min 36s, total: 1h 33min 26s
Wall time: 16min 4s
CPUs: 4
peak memory: 2345.35 MiB, increment: 1441.85 MiB
CPU times: user 1h 16min 28s, sys: 14min 21s, total: 1h 30min 50s
Wall time: 15min 36s
CPUs: 8
peak memory: 2257.62 MiB, increment: 1330.08 MiB
CPU times: user 1h 19min 40s, sys: 14min 18s, total: 1h 33min 59s
Wall time: 16min 5s


In [30]:
p = 14.62 / 14.73 = .99

SyntaxError: can't assign to operator (<ipython-input-30-dae6afadcd8a>, line 1)

What sort of speedup can we potentially get with joblib? 

With N=8 on my laptop, the speedup as follows:
Speedup = 1/((1-p) + p/N) = 1/((1-p) + p/8) = 